In [ ]:
import os
from PIL import Image
import depth_pro
import pillow_heif  
import numpy as np
import matplotlib.pyplot as plt

pillow_heif.register_heif_opener()

model, transform = depth_pro.create_model_and_transforms()
model.eval()

image_dir = r"E:\ITP\itp2\SGFood"  
print(image_dir)

total_img_count = 0
depth_img_count = 0
processed_img_count = 0

def format_depth(depth_img):
    """
    Function to invert the colormap back to the original or inverted form.
    """
    # Convert the image to a numpy array for manipulation
    depth_img_np = np.array(depth_img.cpu())

    # Normalize the depth image to the range 0-1 for displaying
    normalized_depth = (depth_img_np - depth_img_np.min()) / (depth_img_np.max() - depth_img_np.min())

    # Apply a colormap (jet by default)
    colormap = plt.cm.jet
    colormap_image = colormap(normalized_depth)

    # Convert the colormap image from RGBA to RGB
    colormap_image_rgb = (colormap_image[:, :, :3] * 255).astype(np.uint8)

    # Convert the numpy array back to a PIL image
    depth_img = Image.fromarray(colormap_image_rgb)
    return depth_img

# Supported image file extensions
supported_formats = (".jpg", ".jpeg", ".png", ".jfif", ".heic", ".JPG")

# Traverse through all subdirectories and files
for root, dirs, files in os.walk(image_dir):
    # Skip creating multiple depth folders; create it only once per original folder
    if any(file.lower().endswith(supported_formats) for file in files):
        # Create a corresponding depth folder for each directory with the `_depth` suffix
        depth_folder = f"{root}_depth"

        # Create the depth folder if it does not exist
        os.makedirs(depth_folder, exist_ok=True)  
    
        # Process each file in the current directory
        for file_name in files:
            # Skip files that are already depth images
            if "_depth" in file_name.lower():
                print("Skipping depth image:", file_name)
                continue

            # Check if the file is in one of the supported formats
            if file_name.lower().endswith(supported_formats):
                total_img_count += 1  # Count every image encountered
                image_path = os.path.join(root, file_name)  # Original image path
                
                # Construct the depth image filename with `_depth` suffix
                depth_file_name = f"{os.path.splitext(file_name)[0]}_depth.png"
                depth_path = os.path.join(depth_folder, depth_file_name)  # Path for the depth image
                
                # Check if the depth file already exists
                if os.path.exists(depth_path):
                    print("Depth image already exists:", depth_path)
                    depth_img_count += 1
                else:
                    print("Depth image not found, processing:", depth_path)
                    
                    # Load the image (Pillow can now open .heic images with pillow_heif)
                    image = Image.open(image_path)
                    if image.mode == 'RGBA':
                        image = image.convert("RGB")  # Convert to RGB if the image has an alpha channel
                    
                    # Continue processing the image
                    processed_img_count += 1

                    # Preprocess the image if required by the model
                    image = transform(image)  # Apply any preprocessing needed

                    # Run model inference
                    prediction = model.infer(image)  # Adjust based on actual method
                    depth_img = prediction["depth"]  # Get depth data
                    
                    # Format and save depth image
                    depth_img = format_depth(depth_img)  # Format depth data for saving
                    depth_img.save(depth_path)  # Save depth image with modified name
                    print("Saved depth image:", depth_path)

# Print summary statistics
print(f"Total images encountered: {total_img_count}")
print(f"Total existing depth images: {depth_img_count}")
print(f"Total images processed to create new depth images: {processed_img_count}")


E:\ITP\itp2\SGFood
Depth image already exists: E:\ITP\itp2\SGFood\Ang_Ku_Kueh_depth\Ang_Ku_Kueh_1_depth.png
Depth image already exists: E:\ITP\itp2\SGFood\Ang_Ku_Kueh_depth\Ang_Ku_Kueh_2_depth.png
Depth image already exists: E:\ITP\itp2\SGFood\Ang_Ku_Kueh_depth\Ang_Ku_Kueh_3_depth.png
Skipping depth image: Ang_Ku_Kueh_1_depth.png
Skipping depth image: Ang_Ku_Kueh_2_depth.png
Skipping depth image: Ang_Ku_Kueh_3_depth.png
Depth image already exists: E:\ITP\itp2\SGFood\Ayam_Panggang_depth\Ayam_Panggang_1_depth.png
Depth image already exists: E:\ITP\itp2\SGFood\Ayam_Panggang_depth\Ayam_Panggang_10_depth.png
Depth image already exists: E:\ITP\itp2\SGFood\Ayam_Panggang_depth\Ayam_Panggang_11_depth.png
Depth image already exists: E:\ITP\itp2\SGFood\Ayam_Panggang_depth\Ayam_Panggang_12_depth.png
Depth image already exists: E:\ITP\itp2\SGFood\Ayam_Panggang_depth\Ayam_Panggang_13_depth.png
Depth image already exists: E:\ITP\itp2\SGFood\Ayam_Panggang_depth\Ayam_Panggang_14_depth.png
Depth image a

: 